In [1]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [3]:
%matplotlib

Using matplotlib backend: <object object at 0x0000015F6C9B1A50>


In [30]:
#model response
m = 0.023
R = 0.02
g = 9.8
L = 0.3
d = 0.12
J = (2/3)*m*R**2
s = ct.tf('s')
G = ((m*g*d)/(L*((J/R**2)+m)))*(1/s**2)
print(G)

Kp = 85
Ki = 212.585
Kd = 8.5034
Kcp = 85
Tip = 0.4
Tdp = 0.1
C = Kp+Ki/s+Kd*s
CS = Kcp*(1+1/(Tip*s)+Tdp*s)

plant = ct.feedback(G*CS)
print(plant)
T,U = ct.step_response(15*plant)
fig8,ax8 = plt.subplots()

L = len(U)
zeros = []
for i in range(0,L-1):
    if abs((U[-1]-U[i-1])/U[-1])>0.05 and abs((U[-1]-U[i+1])/U[-1])<0.05:
        zeros.append(T[i])
for i in range(0,L):
    if (U[-1]-U[i])/U[-1]<0.05 and T[i]>=zeros[-1]:
        I = i
        sett=T[i]
        Usett = U[i]
        break

dict = ct.step_info(15*plant)
print(dict)
over = dict['Overshoot']
print('Overshoot = ',over,'%')
rise = dict['RiseTime']
print('Rise time =',rise,'s')
print("Settling time =",sett,'s')

ax8.plot(T,U,label = 'response')
ax8.axhline(15,color = 'red',linestyle ='dotted',label = 'setpoint')
ax8.axhline(U[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax8.axhline(U[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax8.grid()
ax8.plot(sett,Usett,'*',label = "settling point",color = 'r')
#ax8.vlines(0.45,ymin=15,ymax = U[-1],color='r',label = "steady state error")
ax8.set_ylabel('Displacement [cm]',fontsize=15)
ax8.set_xlabel('Time [s]',fontsize=15)
ax8.vlines(T[np.argmax(U)],ymin = 15,ymax=np.max(U),color = 'r',label = 'overshoot')
fig8.suptitle('SIMC Tuned Model Step Response for Squash Ball',fontsize=15)
ax8.legend()
ticks = np.linspace(0,19,20)
ax8.set_yticks(ticks)


2.352
-----
 s^2


     7.997 s^2 + 79.97 s + 199.9
-------------------------------------
0.4 s^3 + 7.997 s^2 + 79.97 s + 199.9

{'RiseTime': 0.05029258606074808, 'SettlingTime': 0.6705678141433078, 'SettlingMin': 13.938665529677278, 'SettlingMax': 18.884191279194457, 'Overshoot': 25.894608527963047, 'Undershoot': 0, 'Peak': 18.884191279194457, 'PeakTime': 0.16764195353582695, 'SteadyStateValue': 15.0}
Overshoot =  25.894608527963047 %
Rise time = 0.05029258606074808 s
Settling time = 0.3352839070716539 s


In [26]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figz,axz = plt.subplots(5,1,figsize = (6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)

fig.suptitle('Individual SIMC Tuned Responses for Squash Ball')
figraw.suptitle('Unfiltered SIMC Tuned Responses for Squash Ball')
figz.suptitle('Motor Angle for SIMC Tuned Squash Ball Response')

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of SIMC Tuned Squash Ball Responses')
#fig3.suptitle('Unfiltered Average of Manually Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Distance [cm]')
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1057
for i in range(1,6):
    df = pd.read_csv('SIMC_PID_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs[i-1].plot(timenew,filtered,label = 'filtered',color = 'b')
    axs[i-1].plot(timenew,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1].set_xlabel('Time [s]')
    axs[i-1].set_ylabel('Distance [cm]')
    axs[i-1].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1].grid()
    
    axsraw[i-1].plot(timenew,distance)
    axsraw[i-1].set_xlabel('Time [s]')
    axsraw[i-1].set_ylabel('Distance [cm]')
    axsraw[i-1].grid()
    
    axz[i-1].plot(timenew,angle_filt,label = 'filtered',color = 'b')
    axz[i-1].plot(timenew,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1].set_xlabel('Time [s]')
    axz[i-1].set_ylabel('Servo Angle [degrees]')
    axz[i-1].grid()
    axz[i-1].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axs[0].legend(loc = 'lower right')
axz[0].legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<2.5 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,26,27)
ax2.set_yticks(ticks)
#ax3.plot(time_ave,dist_ave,label = 'response')

1057
1633
1351
1469
1415
[0.014000000000000002, 0.0312, 0.0494, 0.0664, 0.08460000000000001, 0.1018, 0.11939999999999999, 0.137, 0.154, 0.17239999999999997, 0.18960000000000005, 0.20779999999999998, 0.22480000000000003, 0.24219999999999997, 0.26020000000000004, 0.2774, 0.29579999999999995, 0.3128, 0.32999999999999996, 0.34819999999999995, 0.3652, 0.3834000000000001, 0.40040000000000003, 0.41819999999999996, 0.43579999999999997, 0.4532, 0.47119999999999995, 0.48860000000000003, 0.5065999999999999, 0.5238000000000002, 0.541, 0.5596, 0.5768000000000001, 0.5953999999999999, 0.6132, 0.631, 0.6492, 0.6668000000000001, 0.6858000000000001, 0.7031999999999999, 0.7222, 0.7396, 0.758, 0.7767999999999999, 0.7944, 0.8134, 0.8311999999999999, 0.8501999999999998, 0.8682000000000001, 0.8870000000000001, 0.9056000000000001, 0.9232000000000001, 0.942, 0.96, 0.9785999999999999, 0.9965999999999999, 1.0146, 1.033, 1.0508, 1.0694, 1.0876000000000001, 1.1064, 1.1238, 1.1423999999999999, 1.161, 1.178999999999

In [53]:
# fig,axs = plt.subplots(5,1,figsize=(6,9))
# plt.subplots_adjust(top = 0.95,bottom = 0.05)
# figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
# plt.subplots_adjust(top = 0.95,bottom = 0.05)

# fig.suptitle('Filtered SIMC Tuned Responses')
# figraw.suptitle('Unfiltered SIMC Tuned Responses')

# fig2,ax2 = plt.subplots()
# fig3,ax3 = plt.subplots()
# fig2.suptitle('Filtered Average of SIMC Tuned Responses')
# fig3.suptitle('Unfiltered Average of SIMC Tuned Responses')
# ax2.set_xlabel('Time [s]')
# ax2.set_ylabel('Distance [cm]')
# ax3.set_xlabel('Time [s]')
# ax3.set_ylabel('Distance [cm]')

# time_1 = []
# time_2 = []
# time_3 = []
# time_4 = []
# time_5 = []
# distance_1 = []
# distance_2 = []
# distance_3 = []
# distance_4 = []
# distance_5 = []
# time_ave = []
# dist_ave = []

# sos = signal.butter(3, 1000, 'lp', fs=50000, output='sos')

# for i in range(1,6):
#     df = pd.read_csv('SIMC_PID_{}.csv'.format(i))
    
#     time = (df['time']-5000)/1000
#     distance = df['distance']
#     filtered = signal.sosfiltfilt(sos,distance)
    
#     axs[i-1].plot(time,filtered)
#     axs[i-1].set_xlabel('Time [s]')
#     axs[i-1].set_ylabel('Distance [cm]')
    
#     axsraw[i-1].plot(time,distance)
#     axsraw[i-1].set_xlabel('Time [s]')
#     axsraw[i-1].set_ylabel('Distance [cm]')
    
#     time_name =f"time_{i}"
#     globals()[time_name]=((time[0:1057].to_numpy()))
#     distance_name = f"distance_{i}"
#     globals()[distance_name]=distance[0:1057].to_numpy()

# for i in range(0,1056):
#     time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
#     dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)

# filtered = signal.sosfiltfilt(sos,dist_ave)
# ax2.plot(time_ave,filtered)
# ax3.plot(time_ave,dist_ave)

In [ ]:
#fig,axs = plt.subplots(5,1,figsize=(6,9))
#fig.suptitle('SIMC Tuned Responses')
#plt.subplots_adjust(top = 0.95,bottom = 0.05)
#for i in range(0,5):
#    print(i)
#    df = pd.read_csv('SIMC_PID_{}.csv'.format(i+1))
#    time = (df['time']-5000)/1000
#    distance = df['distance']
#    axs[i].plot(time,distance)
#    axs[i].set_xlabel('Time [s]')
#    axs[i].set_ylabel('Distance [cm]')

In [44]:
#Filter
#sos = signal.butter(3, 1000, 'lp', fs=50000, output='sos')
#t = time_ave
#filtered = signal.sosfiltfilt(sos,dist_ave)
#fig3,ax3 = plt.subplots()
#ax3.plot(t,filtered)

In [34]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual SIMC Tuned Responses for Squash Ball',fontsize=20)
figraw.suptitle('Unfiltered SIMC Tuned Responses for Squash Ball',fontsize=20)
figz.suptitle('Motor Angle for SIMC Tuned Squash Ball Response',fontsize=20)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of SIMC Tuned Squash Ball Responses',fontsize=15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize=15)
ax2.set_ylabel('Distance [cm]',fontsize=15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []


lines = ['solid','dashed','dotted','dashdot','solid']

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1500, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1057
for i in range(1,6):
    df = pd.read_csv('SIMC_PID_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]',fontsize=15)
    axs.set_ylabel('Distance [cm]',fontsize=15)
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize=15)
    axsraw.set_ylabel('Distance [cm]',fontsize=15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize=15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize=15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right',fontsize='15')
axz.legend(loc = 'lower right',fontsize='15')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<2.5 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,26,27)
ax2.set_yticks(ticks)
#ax3.plot(time_ave,dist_ave,label = 'response')

1057
1633
1351
1469
1415
[0.014000000000000002, 0.0312, 0.0494, 0.0664, 0.08460000000000001, 0.1018, 0.11939999999999999, 0.137, 0.154, 0.17239999999999997, 0.18960000000000005, 0.20779999999999998, 0.22480000000000003, 0.24219999999999997, 0.26020000000000004, 0.2774, 0.29579999999999995, 0.3128, 0.32999999999999996, 0.34819999999999995, 0.3652, 0.3834000000000001, 0.40040000000000003, 0.41819999999999996, 0.43579999999999997, 0.4532, 0.47119999999999995, 0.48860000000000003, 0.5065999999999999, 0.5238000000000002, 0.541, 0.5596, 0.5768000000000001, 0.5953999999999999, 0.6132, 0.631, 0.6492, 0.6668000000000001, 0.6858000000000001, 0.7031999999999999, 0.7222, 0.7396, 0.758, 0.7767999999999999, 0.7944, 0.8134, 0.8311999999999999, 0.8501999999999998, 0.8682000000000001, 0.8870000000000001, 0.9056000000000001, 0.9232000000000001, 0.942, 0.96, 0.9785999999999999, 0.9965999999999999, 1.0146, 1.033, 1.0508, 1.0694, 1.0876000000000001, 1.1064, 1.1238, 1.1423999999999999, 1.161, 1.178999999999